# Sedona Experiments

Read in data

In [ ]:
from sedona.spark import SedonaContext
from pyspark.sql.functions import expr

# Start Spark + Sedona
config = SedonaContext.builder(). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all/'). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.5_2.12:1.7.1,'
           'org.datasyslab:geotools-wrapper:1.7.1-28.5,'
           'edu.ucar:cdm-core:5.4.2'). \
    config('spark.driver.memory', '10g'). \
    config('spark.driver.maxResultSize', '5g'). \
    config('spark.network.timeout', '1000s'). \
    config("spark.sql.legacy.parquet.nanosAsLong", "true"). \
    getOrCreate()
sedona = SedonaContext.create(config)

25/05/26 13:16:07 WARN Utils: Your hostname, cs-spatial-501 resolves to a loopback address: 127.0.0.1; using 128.101.33.153 instead (on interface eno1)
25/05/26 13:16:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://artifacts.unidata.ucar.edu/repository/unidata-all/ added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/uribe055/.ivy2/cache
The jars for the packages stored in: /home/uribe055/.ivy2/jars
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
edu.ucar#cdm-core added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4e6169e5-3fac-40c1-9322-64d57026867b;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/uribe055/sedona_experiments/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-3.5_2.12;1.7.1 in central
	found org.apache.sedona#sedona-common;1.7.1 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	found com.uber#h3;4.1.1 in central
	found net.sf.geographiclib#GeographicLib-Java;1.52 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.2 in central
	found org.checkerframework#checker-qual;3.10.0 in central
	found com.google.error

25/05/26 13:16:24 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
sdf = sedona.read.parquet(f"/home/uribe055/sedona_experiments/processed_data/*.parquet")

Create a geometry column for spatial queries

In [4]:
sdf = sdf.withColumn("geom", expr("ST_Point(longitude, latitude)")).drop("number", "expver")
sdf.createOrReplaceTempView("GL")
sdf.show()

+--------+---------+-------------------+--------------+-----------------+
|latitude|longitude|               time|2m_temperature|             geom|
+--------+---------+-------------------+--------------+-----------------+
|    84.0|    -75.0|2023-12-27 00:00:00|     243.76312|   POINT (-75 84)|
|    84.0|   -74.75|2023-12-27 00:00:00|     243.75336|POINT (-74.75 84)|
|    84.0|    -74.5|2023-12-27 00:00:00|     243.74554| POINT (-74.5 84)|
|    84.0|   -74.25|2023-12-27 00:00:00|     243.73773|POINT (-74.25 84)|
|    84.0|    -74.0|2023-12-27 00:00:00|     243.72015|   POINT (-74 84)|
|    84.0|   -73.75|2023-12-27 00:00:00|     243.66351|POINT (-73.75 84)|
|    84.0|    -73.5|2023-12-27 00:00:00|     243.60687| POINT (-73.5 84)|
|    84.0|   -73.25|2023-12-27 00:00:00|     243.55023|POINT (-73.25 84)|
|    84.0|    -73.0|2023-12-27 00:00:00|     243.49554|   POINT (-73 84)|
|    84.0|   -72.75|2023-12-27 00:00:00|      243.4389|POINT (-72.75 84)|
|    84.0|    -72.5|2023-12-27 00:00:0

### Changing Spatial and Temporal Resolution

In [ ]:
# def daily_05(start_time, end_time, min_lat,max_lat,min_lon,max_lon):
#     query = f"""
#         SELECT time, int(latitude/0.5)*0.5, int(longitude/0.5)*0.5, avg(2m_temperature)
#         FROM GL
#         WHERE latitude BETWEEN {min_lat} AND {max_lat}
#             AND longitude BETWEEN {min_lon} AND {max_lon}
#             AND time BETWEEN '{start_time}' AND '{end_time}'
#         GROUP BY time, int(latitude/0.5), int(longitude/0.5)
#         """
#     return query

# def daily_1(start_time, end_time, min_lat,max_lat,min_lon,max_lon):
#     query = f"""
#         SELECT time, int(latitude), int(longitude), avg(2m_temperature)
#         FROM GL
#         WHERE latitude BETWEEN {min_lat} AND {max_lat}
#             AND longitude BETWEEN {min_lon} AND {max_lon}
#             AND time BETWEEN '{start_time}' AND '{end_time}'
#         GROUP BY time, int(latitude), int(longitude)
#         """
#     return query

In [23]:
def daily(s, start_time, end_time, min_lat, max_lat, min_lon, max_lon):
    q1 = f"""
        FROM GL
        WHERE latitude BETWEEN {min_lat} AND {max_lat}
            AND longitude BETWEEN {min_lon} AND {max_lon}
            AND time BETWEEN '{start_time}' AND '{end_time}'
        """
    
    if s == 0.25:
        q0 = f"""SELECT time, latitude, longitude, avg(2m_temperature)""" 
        q2 = """GROUP BY time, latitude, longitude"""
    elif s == 0.5:
        q0 = f"""SELECT time, int(latitude/0.5)*0.5, int(longitude/0.5)*0.5, avg(2m_temperature)"""
        q2 = f"""GROUP BY time, int(latitude/0.5), int(longitude/0.5)"""
    else:
        q0 = f"""SELECT time, int(latitude), int(longitude), avg(2m_temperature)"""
        q2 = f"""GROUP BY time, int(latitude), int(longitude)"""

    return q0 + q1 + q2

In [25]:
def monthly(s, start_time, end_time, min_lat, max_lat, min_lon, max_lon):
    q1 = f"""
        FROM GL
        WHERE latitude BETWEEN {min_lat} AND {max_lat}
            AND longitude BETWEEN {min_lon} AND {max_lon}
            AND time BETWEEN '{start_time}' AND '{end_time}'
        """
    
    if s == 0.25:
        q0 = f"""SELECT month(time), latitude, longitude, avg(2m_temperature)""" 
        q2 = """GROUP BY month(time), latitude, longitude"""
    elif s == 0.5:
        q0 = f"""SELECT month(time), int(latitude/0.5)*0.5, int(longitude/0.5)*0.5, avg(2m_temperature)"""
        q2 = f"""GROUP BY month(time), int(latitude/0.5), int(longitude/0.5)"""
    else:
        q0 = f"""SELECT month(time), int(latitude), int(longitude), avg(2m_temperature)"""
        q2 = f"""GROUP BY month(time), int(latitude), int(longitude)"""

    return q0 + q1 + q2

In [28]:
def yearly(s, start_time, end_time, min_lat, max_lat, min_lon, max_lon):
    q1 = f"""
        FROM GL
        WHERE latitude BETWEEN {min_lat} AND {max_lat}
            AND longitude BETWEEN {min_lon} AND {max_lon}
            AND time BETWEEN '{start_time}' AND '{end_time}'
        """
    
    if s == 0.25:
        q0 = f"""SELECT year(time), latitude, longitude, avg(2m_temperature)""" 
        q2 = """GROUP BY year(time), latitude, longitude"""
    elif s == 0.5:
        q0 = f"""SELECT year(time), int(latitude/0.5)*0.5, int(longitude/0.5)*0.5, avg(2m_temperature)"""
        q2 = f"""GROUP BY year(time), int(latitude/0.5), int(longitude/0.5)"""
    else:
        q0 = f"""SELECT year(time), int(latitude), int(longitude), avg(2m_temperature)"""
        q2 = f"""GROUP BY year(time), int(latitude), int(longitude)"""

    return q0 + q1 + q2

In [29]:
result = sedona.sql(yearly(1.0, "2020-01-01 00:00:00","2024-12-31 23:00:00",60,85,-69,-10,))
result.show()

+----------+--------+---------+-------------------+
|year(time)|latitude|longitude|avg(2m_temperature)|
+----------+--------+---------+-------------------+
|      2023|      81|      -58| 255.45603577078205|
|      2023|      80|      -49| 252.20316722033775|
|      2023|      79|      -38| 248.29399140501675|
|      2023|      78|      -30| 252.37032335359757|
|      2023|      70|      -32|  249.6472076546656|
|      2023|      64|      -34| 277.59021756942957|
|      2023|      62|      -28| 280.13111580626605|
|      2021|      71|      -40|   247.917165521073|
|      2021|      70|      -10|  275.0783130384471|
|      2021|      68|      -22|  272.2759355100867|
|      2021|      64|      -55|  273.0838429986614|
|      2021|      64|      -25|  279.1653110451894|
|      2020|      81|      -22| 255.20839943390726|
|      2020|      80|      -46|  249.5908606221767|
|      2020|      77|      -64| 256.07421290548774|
|      2020|      76|      -27| 251.33068185034998|
|      2020|

Experiment Queries:

In [ ]:
t_res = ["hour","hour","hour", "day","day","day", "year","year","year", "month","month"]
s_res = [0.25, 0.5, 1.0, 0.25, 0.5, 1.0, 0.25, 0.5, 1.0, 0.5, 1.0]

start_time = "2020-01-01 00:00:00"
end_time = "2024-12-31 23:00:00"
min_lat = 60
max_lat = 80
min_lon = -60
max_lon = -30

results_list = []
for t, s in zip(t_res, s_res):
    if t == "hour":
        query = """
                """
    elif t == "day":
        query = daily(s, start_time,end_time,min_lat,max_lat,min_lon,max_lon)
    elif t == "month":
        query = monthly(s, start_time,end_time,min_lat,max_lat,min_lon,max_lon)
    else:
        query = yearly(s, start_time,end_time,min_lat,max_lat,min_lon,max_lon)

    query_start = time.time()
    result = sedona.sql(query)
    query_end = time.time()

    results_list.append({"sys": "Sedona", 
                         "t_res": t,
                         "s_res": s,
                         "time_span":q["time_span"],             # FOR HEATMAP
                         "tr": None,
                         "ta": None,
                         "total_time": query_end - query_start,
                         "percent_area":100     # FOR CHANGING RESULT SIZE
                        })

### Changing Result Size

In [ ]:
res_pairs = [("hour", 0.25), ("year", 0.25), ("month", 0.5), ("hour", 1.0), ("year", 1.0)]

### Heatmap Query Performance

In [ ]:
time_span = [1, 2.5, 5]

### Find Time Query Performance